In [ ]:
  pip install keras-multi-head


In [ ]:
import keras
from keras_multi_head import MultiHeadAttention
from keras_multi_head import MultiHead
!pip install keras-self-attention
!pip install keras-multi-head
import tensorflow as tf
import keras
from keras_self_attention import SeqSelfAttention
from sklearn.metrics import fbeta_score
from keras import backend as K

In [ ]:
from keras_self_attention import SeqSelfAttention

In [ ]:
import pickle 
import numpy as np


In [ ]:
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2
    y_pred = K.clip(y_pred, 0, 1)
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [ ]:
from keras_multi_head import MultiHead

In [ ]:

@tf.function
def fastGELU(x):
    return tf.math.maximum(0.,tf.math.minimum(1.,((1.702*x+1)/2)))*x



inputs_audio =tf.keras.Input(shape=(None,39))
inputs_text=tf.keras.Input(shape=(None,768))
normalised_audio=tf.keras.layers.BatchNormalization()(inputs_audio)
normalised_text=tf.keras.layers.BatchNormalization()(inputs_text)
mask1=tf.keras.layers.Masking()(normalised_audio)
mask2=tf.keras.layers.Masking()(normalised_text)
gru_audio = keras.layers.GRU(units=78,recurrent_dropout=0.3,return_sequences=True)(mask1)
gru_text = keras.layers.GRU(units=192,recurrent_dropout=0.3,return_sequences=True)(mask2)
selfatt_audio =SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       )(gru_audio)
selfatt_text =  SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       )(gru_text)
att_layer = MultiHeadAttention(head_num=2)([selfatt_text,selfatt_audio,selfatt_audio])
gap=tf.keras.layers.GlobalAveragePooling1D()(gru_audio)
fc1=tf.keras.layers.Dense(80,activation=fastGELU)(gap)
norm1=tf.keras.layers.BatchNormalization()(fc1)
d1=tf.keras.layers.Dropout(0.3)(norm1)
fc2=tf.keras.layers.Dense(80,activation=fastGELU)(d1)
norm2=tf.keras.layers.BatchNormalization()(fc2)
d2=tf.keras.layers.Dropout(0.3)(norm2)
result=tf.keras.layers.Dense(4,activation=tf.keras.activations.softmax)(d2)
model = keras.models.Model(inputs=[inputs_audio,inputs_text], outputs=[result])

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3,decay=1e-6),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision(),fbeta]
)
model.summary(line_length=100)


Model: "functional_1"
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             [(None, None, 39)]    0                                            
____________________________________________________________________________________________________
batch_normalization_4 (BatchNorm (None, None, 39)      156         input_5[0][0]                    
____________________________________________________________________________________________________
masking_4 (Masking)              (None, None, 39)      0           batch_normalization_4[0][0]      
____________________________________________________________________________________________________
gru_4 (GRU)                      (None, None, 78)      27846       masking_4[0][0]                  
_____________________________________________________________________

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
x_train=pickle.load(open('/content/drive/My Drive/data/impro1trainx.pickle',"rb"))
y_train=pickle.load(open('/content/drive/My Drive/data/impro1trainy.pickle',"rb"))
x_test=pickle.load(open('/content/drive/My Drive/data/impro1testx.pickle',"rb"))
y_test=pickle.load(open('/content/drive/My Drive/data/impro1testy.pickle',"rb"))


In [ ]:
max_train=0
for i in x_train:
  if (max_train<np.shape(i[0])[1]):
    max_train=np.shape(i[0])[1]
print(max_train)
for i in x_train:
  i[0]=np.transpose(i[0])
  result=np.zeros((max_train,39))
  result[:np.shape(i[0])[0],:]=i[0]
  i[0]=result
  i[0]=tf.convert_to_tensor(i[0])
  i[1]=tf.convert_to_tensor(i[1])


1522


In [ ]:
audio=[]
text=[]
for i in x_train:
  audio.append(i[0])
  text.append(i[1])
audio=tf.convert_to_tensor(audio)
text=tf.squeeze(tf.convert_to_tensor(text))

In [ ]:
x_valid=pickle.load(open('/content/drive/My Drive/data/impro1validx.pickle',"rb"))
y_valid=pickle.load(open('/content/drive/My Drive/data/impro1validy.pickle',"rb"))

In [ ]:
max_train=0
for i in x_train:
  if (max_train<np.shape(i[0])[1]):
    max_train=np.shape(i[0])[1]
  print(np.shape(i[0]))

(39, 88)
(39, 382)
(39, 166)
(39, 91)
(39, 316)
(39, 144)
(39, 841)
(39, 119)
(39, 213)
(39, 163)
(39, 316)
(39, 379)
(39, 88)
(39, 254)
(39, 172)
(39, 88)
(39, 241)
(39, 266)
(39, 316)
(39, 272)
(39, 391)
(39, 213)
(39, 810)
(39, 476)
(39, 479)
(39, 426)
(39, 304)
(39, 357)
(39, 576)
(39, 372)
(39, 1101)
(39, 822)
(39, 797)
(39, 69)
(39, 126)
(39, 582)
(39, 335)
(39, 288)
(39, 535)
(39, 407)
(39, 457)
(39, 454)
(39, 272)
(39, 626)
(39, 429)
(39, 497)
(39, 251)
(39, 444)
(39, 479)
(39, 1038)
(39, 251)
(39, 441)
(39, 116)
(39, 741)
(39, 210)
(39, 110)
(39, 907)
(39, 1119)
(39, 354)
(39, 141)
(39, 304)
(39, 160)
(39, 82)
(39, 447)
(39, 422)
(39, 147)
(39, 585)
(39, 332)
(39, 669)
(39, 394)
(39, 538)
(39, 354)
(39, 907)
(39, 138)
(39, 507)
(39, 135)
(39, 507)
(39, 882)
(39, 310)
(39, 169)
(39, 182)
(39, 113)
(39, 104)
(39, 88)
(39, 341)
(39, 179)
(39, 76)
(39, 82)
(39, 69)
(39, 194)
(39, 116)
(39, 69)
(39, 347)
(39, 85)
(39, 104)
(39, 507)
(39, 316)
(39, 176)
(39, 69)
(39, 347)
(39, 116)


In [ ]:
model.fit(x=[audio,text],y=y_train,epochs=50,batch_size=30)

In [ ]:
max_test=0
for i in x_test:
  if (max_test<np.shape(i[0])[0]):
    max_test=np.shape(i[0])[0]
 

(1, 13, 432)
(360, 13)
(345, 13)
(294, 13)
(219, 13)
(252, 13)
(192, 13)
(279, 13)
(282, 13)
(276, 13)
(72, 13)
(132, 13)
(123, 13)
(237, 13)
(123, 13)
(78, 13)
(294, 13)
(234, 13)
(108, 13)
(285, 13)
(276, 13)
(159, 13)
(99, 13)
(132, 13)
(234, 13)
(99, 13)
(195, 13)
(105, 13)
(276, 13)
(198, 13)
(180, 13)
(69, 13)
(66, 13)
(159, 13)
(102, 13)
(225, 13)
(138, 13)
(204, 13)
(459, 13)
(411, 13)
(177, 13)
(168, 13)
(363, 13)
(78, 13)
(114, 13)
(582, 13)
(138, 13)
(273, 13)
(129, 13)
(189, 13)
(141, 13)
(159, 13)
(72, 13)
(171, 13)
(162, 13)
(93, 13)
(87, 13)
(90, 13)
(243, 13)
(102, 13)
(351, 13)
(108, 13)
(348, 13)
(366, 13)
(1215, 13)
(165, 13)
(84, 13)
(84, 13)
(180, 13)
(492, 13)
(381, 13)
(90, 13)
(78, 13)
(132, 13)
(438, 13)
(378, 13)
(81, 13)
(342, 13)
(102, 13)
(222, 13)
(387, 13)
(270, 13)
(135, 13)
(390, 13)
(378, 13)
(195, 13)
(384, 13)
(84, 13)
(360, 13)
(78, 13)
(99, 13)
(87, 13)
(102, 13)
(123, 13)
(78, 13)
(93, 13)
(198, 13)
(147, 13)
(165, 13)
(75, 13)
(117, 13)
(189, 13)

In [ ]:
max_train

1522

In [ ]:
max_valid=0
for i in x_valid:
  if (max_valid<np.shape(i[0])[0]):
    max_valid=np.shape(i[0])[0]

In [ ]:
result = np.zeros((1,max_train,39))
  result[:,:i[0].shape[1],:i[0].shape[2]] = i[0]
  i[0]=result
  i[0]=tf.convert_to_tensor(i[0])
  i[1]=tf.convert_to_tensor(i[1])

13

In [ ]:
for i in x_train:
  i[0]=np.transpose(i[0])
  i[0]=np.reshape(i[0],(1,)+np.shape(i[0]))
  print(np.shape(i[0]))
  


(1, 1, 39, 88, 1, 1)
(1, 382, 39)
(1, 166, 39)
(1, 91, 39)
(1, 316, 39)
(1, 144, 39)
(1, 841, 39)
(1, 119, 39)
(1, 213, 39)
(1, 163, 39)
(1, 316, 39)
(1, 379, 39)
(1, 88, 39)
(1, 254, 39)
(1, 172, 39)
(1, 88, 39)
(1, 241, 39)
(1, 266, 39)
(1, 316, 39)
(1, 272, 39)
(1, 391, 39)
(1, 213, 39)
(1, 810, 39)
(1, 476, 39)
(1, 479, 39)
(1, 426, 39)
(1, 304, 39)
(1, 357, 39)
(1, 576, 39)
(1, 372, 39)
(1, 1101, 39)
(1, 822, 39)
(1, 797, 39)
(1, 69, 39)
(1, 126, 39)
(1, 582, 39)
(1, 335, 39)
(1, 288, 39)
(1, 535, 39)
(1, 407, 39)
(1, 457, 39)
(1, 454, 39)
(1, 272, 39)
(1, 626, 39)
(1, 429, 39)
(1, 497, 39)
(1, 251, 39)
(1, 444, 39)
(1, 479, 39)
(1, 1038, 39)
(1, 251, 39)
(1, 441, 39)
(1, 116, 39)
(1, 741, 39)
(1, 210, 39)
(1, 110, 39)
(1, 907, 39)
(1, 1119, 39)
(1, 354, 39)
(1, 141, 39)
(1, 304, 39)
(1, 160, 39)
(1, 82, 39)
(1, 447, 39)
(1, 422, 39)
(1, 147, 39)
(1, 585, 39)
(1, 332, 39)
(1, 669, 39)
(1, 394, 39)
(1, 538, 39)
(1, 354, 39)
(1, 907, 39)
(1, 138, 39)
(1, 507, 39)
(1, 135, 39)
(1, 50

In [ ]:
for i in x_train:
  result = np.zeros((1,max_train,39))
  result[:,:np.shape(i[0])[1],:np.shape(i[0])[2]] = i[0]
  i[0]=result
  i[0]=tf.convert_to_tensor(i[0])
  i[1]=tf.convert_to_tensor(i[1])


In [ ]:
for i in x_train:
  i[0]=np.transpose(i[0])
  i[0]=np.reshape(i[0],(1,)+np.shape(i[0]))
  result = np.zeros((1,max_train,39))
  result[:,:i[0].shape[1],:i[0].shape[2]] = i[0]
  i[0]=result
  i[0]=tf.convert_to_tensor(i[0])
  i[1]=tf.convert_to_tensor(i[1])
  


In [ ]:
for i in x_valid:
  i[0]=np.transpose(i[0])
  i[0]=np.reshape(i[0],(1,)+np.shape(i[0]))
  result = np.zeros((1,max_valid,39))
  result[:,:i[0].shape[1],:i[0].shape[2]] = i[0]
  i[0]=result
  i[0]=tf.convert_to_tensor(i[0])
  i[1]=tf.convert_to_tensor(i[1])

In [ ]:
audio_train=[]
text_train=[]
for i in x_train:
  audio_train.append(i[0])
  text_train.append(i[1])
audio_train=tf.squeeze(tf.convert_to_tensor(audio_train))
text_train=tf.squeeze(tf.convert_to_tensor(text_train))
y_train=tf.squeeze(tf.convert_to_tensor(y_train))

In [ ]:
audio_test=[]
text_test=[]
for i in x_test:
  audio_test.append(i[0])
  text_test.append(i[1])
audio_test=tf.squeeze(tf.convert_to_tensor(audio_test))
text_test=tf.squeeze(tf.convert_to_tensor(text_test))
y_test=tf.squeeze(tf.convert_to_tensor(y_test))

In [ ]:
audio_valid=[]
text_valid=[]
for i in x_valid:
  audio_valid.append(i[0])
  text_valid.append(i[1])
audio_valid=tf.squeeze(tf.convert_to_tensor(audio_valid))
text_valid=tf.squeeze(tf.convert_to_tensor(text_valid))
y_valid=tf.squeeze(tf.convert_to_tensor(y_valid))

In [ ]:
len(audio_test)

In [ ]:
y_train=tf.squeeze(y_train)

In [ ]:
model.fit(x=[audio,text],y=y_train,batch_size=30)

In [ ]:
model.evaluate(x=[audio_test,text_test],y=y_test)

In [ ]:
y_pred=model.predict(([audio_test,text_test]))

In [ ]:
y_pred

In [ ]:
emotions={}
emotions[tf.convert_to_tensor([1.,0.,0.,0.])]=0
emotions[tf.convert_to_tensor([0.,1.,0.,0.])]=1
emotions[tf.convert_to_tensor([0.,0.,1.,0.])]=2
emotions[tf.convert_to_tensor([0.,0.,0.,1.])]=3

In [ ]:
audio=[]
text=[]
for i in x_train111:
  audio.append(i[0])
  text.append(i[1])

In [ ]:
model.fit_generator(gener,)

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
for i in x_train:
  i=tf.convert_to_tensor(i)

In [ ]:
for i in x_test:
  i[0]=np.transpose(i[0])
  i[0]=np.reshape(i[0],(1,)+np.shape(i[0]))
  i[0]=tf.convert_to_tensor(i[0])
  i[1]=tf.convert_to_tensor(i[1])

In [ ]:
for i in x_valid:
  i[0]=np.transpose(i[0])
  i[0]=np.reshape(i[0],(1,)+np.shape(i[0]))
  i[0]=tf.convert_to_tensor(i[0])
  i[1]=tf.convert_to_tensor(i[1])

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
y_valid

In [ ]:
size=len(x_train)

In [ ]:
mult=size/30

In [ ]:
mult

In [ ]:
mult=int(mult)

In [ ]:
mult

In [ ]:
count=0
for i in range(0,mult):
  audio=[]
  text=[]
  train=[]
  for j in range(i*30,(i+1)*30):
    audio.append(x_train[j][0])
    text.append(x_train[j][1])
    train.append(y_train[j])
  audio=tf.convert_to_tensor(audio)
  text=tf.convert_to_tensor(text)
  train=tf.convert_to_tensor(train)
  audio=tf.squeeze(audio)
  text=tf.squeeze(text)
  train=tf.squeeze(train)
  history=model.train_on_batch(x=[audio,text],y=train,callbacks_list)
  print(history)
  print(count)
  count=count+1
  
  
  
 
  

In [ ]:
hist.History

In [ ]:
x=fbeta(y_test,y_pred)

In [ ]:
x=np.array(x)

In [ ]:
x

In [ ]:
audio=[]
text=[]
for i in x_test:
  audio.append(i[0])
  text.append(i[1])


In [ ]:
len(text)

In [ ]:
y_pred=[]
count=0
for i in range(len(audio)):
  y_pred.append(model.predict([audio[i],text[i]]))
  print(count)
  count=count+1


In [ ]:
tf.keras.metrics.categorical_accuracy(y_test,y_pred)

In [ ]:
audio=[]
text=[]
for i in x_train:
  audio.append(i[0])
  text.append(i[1])

In [ ]:
max=0
for i in audio:
  if (max<tf.shape(i)[1]):
    max=tf.shape(i)[1]


In [ ]:

for i in audio:
  i=np.convert(i)
  result=np.zeros((1,max,39))
  result[:,:np.shape(i)[1],:]=i
  i=result


In [ ]:
audio=tf.ragged.stack(audio)

In [ ]:
len(list1)

In [ ]:
  def __iter__(self):
    for item in (self[i] for i in range(len(self))):
      yield item

In [ ]:
for i in audio:
  print(i)